## 추천시스템 (Recommende System)
- 추천시스템은 크게 두가지로 구분가능
    - 컨텐츠 기반 필터링(content-based filtering)
    - 협업 필터링(collaborative filtering)
- 두가지를 조합한 Hybrid 방식도 가능
- 컨텐츠 기반 필터링은 지금까지 사용자의 이전행동과 명시적 피드백을 통해 사용자가 좋아하는 것과 유사한 항목을 추천
- 협업 필터링은 사용자와 항목간의 유사성을 동시에 사용해 추천

### Surprise
- 추천 시스템 개발을 위한 라이브러리
- 다양한 모델과 데이터 제공
- scikit-learn과 유사한 사용 방법

In [1]:
from surprise import SVD
from surprise import Dataset
from surprise.model_selection import cross_validate
from sklearn.decomposition import randomized_svd, non_negative_factorization
import numpy as np

In [3]:
data = Dataset.load_builtin('ml-100k',prompt=False)
data.raw_ratings[:10]
#data 구성
#user/item(movie)/score(rating)/ID

[('196', '242', 3.0, '881250949'),
 ('186', '302', 3.0, '891717742'),
 ('22', '377', 1.0, '878887116'),
 ('244', '51', 2.0, '880606923'),
 ('166', '346', 1.0, '886397596'),
 ('298', '474', 4.0, '884182806'),
 ('115', '265', 2.0, '881171488'),
 ('253', '465', 5.0, '891628467'),
 ('305', '451', 3.0, '886324817'),
 ('6', '86', 3.0, '883603013')]

In [2]:
model=SVD()

In [4]:
cross_validate(model, data, measures=['rmse', 'mae'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9298  0.9357  0.9473  0.9354  0.9308  0.9358  0.0062  
MAE (testset)     0.7328  0.7403  0.7476  0.7373  0.7316  0.7379  0.0058  
Fit time          4.67    4.76    4.80    4.82    5.00    4.81    0.11    
Test time         0.18    0.18    0.16    0.12    0.16    0.16    0.02    


{'test_rmse': array([0.92981198, 0.93565074, 0.94733454, 0.9354167 , 0.93079398]),
 'test_mae': array([0.73275821, 0.74028896, 0.74760156, 0.73725154, 0.73156323]),
 'fit_time': (4.669963836669922,
  4.755825996398926,
  4.798680067062378,
  4.818830728530884,
  4.995053052902222),
 'test_time': (0.18488502502441406,
  0.17613005638122559,
  0.16278696060180664,
  0.1204061508178711,
  0.1611628532409668)}

### 컨텐츠 기반 필터링(Content-based Filtering)
- 컨텐츠 기반 필터링은 이전의 행동과 명시적 피드백을 통해 좋아하는 것과 유사한 항목을 추천
    - 예를들어 내가 지금까지 시청한 영화 목록과 다른 사용자의 시청 목록을 비교해 나와 비슷한 취향의 사용자가 시청한 영화를 추천
- 유사도를 기반으로 추천
- 컨텐츠 기반 필터링은 다음과 같은 장단점이 있다.
    - 장점
        - 많은 수의 사용자를 대상으로 쉽게 확장이 가능
        - 사용자가 관심을 갖지 않던 상품 추천가능
    - 단점
        - 입력 특성을 직접 설계해야 하기 때문에 많은 도메인 지식이 필요
        - 사용자의 기존 관심사항을 기반으로만 추천 가능
        - 콜드 스타트 문제 존재

In [5]:
data = Dataset.load_builtin('ml-100k', prompt=False)
raw_data = np.array(data.raw_ratings, dtype=int)

In [6]:
raw_data

array([[      196,       242,         3, 881250949],
       [      186,       302,         3, 891717742],
       [       22,       377,         1, 878887116],
       ...,
       [      276,      1090,         1, 874795795],
       [       13,       225,         2, 882399156],
       [       12,       203,         3, 879959583]])

In [12]:
raw_data[:,0]-=1
raw_data[:,1]-=1

In [13]:
raw_data

array([[      195,       241,         3, 881250949],
       [      185,       301,         3, 891717742],
       [       21,       376,         1, 878887116],
       ...,
       [      275,      1089,         1, 874795795],
       [       12,       224,         2, 882399156],
       [       11,       202,         3, 879959583]])

In [19]:
n_users = np.max(raw_data[:,0])
n_movies = np.max(raw_data[:,1])
shape = (n_users+1, n_movies+1)
shape

(943, 1682)

In [20]:
adj_matrix = np.ndarray(shape,dtype=int)
for user_id, movie_id, rating, time in raw_data:
    adj_matrix[user_id][movie_id]=1.
adj_matrix

array([[1, 1, 1, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [1, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0]])

In [21]:
my_id, my_vector = 0, adj_matrix[0]
best_match, best_match_id, best_match_vector = -1,-1,[]

for user_id, user_vector in enumerate(adj_matrix):
    if my_id != user_id:
        similarity = np.dot(my_vector, user_vector)
        if similarity > best_match:
            best_match = similarity
            best_match_id = user_id
            best_match_vector = user_vector
            
print('Best Match : {}, Best Match ID : {}'.format(best_match,best_match_id))

Best Match : 183, Best Match ID : 275


In [22]:
recommed_list = []
for i, log in enumerate(zip(my_vector, best_match_vector)):
    log1, log2 = log
    if log1<1. and log2>0.:
        recommed_list.append(i)
print(recommed_list)

[272, 273, 275, 280, 281, 283, 287, 288, 289, 290, 292, 293, 297, 299, 300, 301, 302, 306, 312, 314, 315, 316, 317, 321, 322, 323, 324, 327, 330, 331, 332, 333, 339, 342, 345, 346, 353, 354, 355, 356, 357, 363, 364, 365, 366, 372, 374, 378, 379, 381, 382, 383, 384, 385, 386, 387, 390, 391, 392, 394, 395, 396, 398, 400, 401, 402, 403, 404, 405, 406, 407, 408, 409, 410, 412, 414, 416, 417, 418, 419, 420, 422, 424, 425, 426, 427, 428, 430, 431, 432, 435, 442, 446, 447, 448, 449, 450, 451, 452, 454, 455, 457, 460, 461, 462, 468, 469, 470, 471, 472, 473, 474, 478, 495, 500, 507, 517, 522, 525, 530, 539, 540, 543, 545, 546, 548, 549, 550, 551, 553, 557, 558, 560, 561, 562, 563, 565, 566, 567, 568, 570, 571, 574, 575, 576, 577, 580, 581, 582, 585, 587, 589, 590, 594, 596, 602, 623, 626, 627, 630, 633, 635, 639, 646, 648, 651, 652, 654, 657, 664, 668, 671, 677, 678, 681, 683, 684, 685, 690, 691, 692, 695, 696, 708, 709, 714, 718, 719, 720, 724, 726, 727, 731, 733, 734, 736, 738, 741, 742, 745,

### 유클리드 거리를 사용해 추천

$$euclidean=\ \sqrt{\sum_{d=1}^{D\ }{(A_i-B_i)^2}} $$

- 거리가 가까울 수록(값이 작을 수록) 나와 유사항 사용자

In [23]:
my_id, my_vector = 0, adj_matrix[0]
best_match, best_match_id, best_match_vector = 999,-1,[]

for user_id, user_vector in enumerate(adj_matrix):
    if my_id != user_id:
        euclidean_dist = np.sqrt(np.sum(np.square(my_vector - user_vector)))
        if euclidean_dist < best_match:
            best_match = euclidean_dist
            best_match_id = user_id
            best_match_vector = user_vector
            
print('Best Match : {}, Best Match ID : {}'.format(best_match,best_match_id))

Best Match : 14.832396974191326, Best Match ID : 737


In [24]:
recommed_list = []
for i, log in enumerate(zip(my_vector, best_match_vector)):
    log1, log2 = log
    if log1<1. and log2>0.:
        recommed_list.append(i)
print(recommed_list)

[297, 312, 317, 342, 356, 366, 379, 384, 392, 402, 404, 407, 417, 422, 428, 433, 448, 454, 469, 473, 495, 510, 516, 526, 527, 549, 567, 602, 635, 649, 650, 654, 658, 661, 664, 696, 731, 746, 750, 754, 915, 918, 925, 929, 950, 968, 1015, 1046]


### 코사인 유사도를 사용해 추천

$$cos\theta = \dfrac{A \bullet B}{\rVert A \rVert \times \rVert B \rVert}$$

- 두 벡터가 이루고 있는 각을 계산

In [25]:
def compute_cos_similarity(v1,v2):
    norm1 = np.sqrt(np.sum(np.square(v1)))
    norm2 = np.sqrt(np.sum(np.square(v2)))
    dot = np.dot(v1,v2)
    return dot / (norm1*norm2)

In [26]:
my_id, my_vector = 0, adj_matrix[0]
best_match, best_match_id, best_match_vector = -1,-1,[]

for user_id, user_vector in enumerate(adj_matrix):
    if my_id != user_id:
        cos_similarity = compute_cos_similarity(my_vector, user_vector)
        if cos_similarity > best_match:
            best_match = cos_similarity
            best_match_id = user_id
            best_match_vector = user_vector
            
print('Best Match : {}, Best Match ID : {}'.format(best_match,best_match_id))

Best Match : 0.5278586163659506, Best Match ID : 915


In [27]:
recommed_list = []
for i, log in enumerate(zip(my_vector, best_match_vector)):
    log1, log2 = log
    if log1<1. and log2>0.:
        recommed_list.append(i)
print(recommed_list)

[272, 275, 279, 280, 283, 285, 289, 294, 297, 316, 317, 355, 365, 366, 368, 379, 380, 381, 384, 386, 392, 398, 401, 404, 416, 420, 422, 424, 426, 427, 430, 432, 450, 460, 461, 466, 469, 471, 473, 474, 475, 479, 482, 483, 497, 505, 508, 510, 511, 522, 526, 527, 529, 530, 534, 536, 540, 545, 548, 549, 556, 557, 558, 560, 565, 567, 568, 569, 577, 580, 581, 582, 592, 596, 630, 635, 639, 641, 649, 651, 654, 673, 677, 678, 683, 684, 692, 696, 701, 703, 707, 708, 709, 712, 714, 719, 720, 726, 731, 734, 736, 738, 740, 745, 747, 754, 755, 761, 762, 763, 766, 780, 789, 791, 805, 819, 823, 824, 830, 843, 862, 865, 918, 929, 930, 938, 942, 943, 947, 958, 959, 960, 970, 977, 1004, 1008, 1009, 1010, 1013, 1041, 1045, 1069, 1072, 1073, 1078, 1097, 1100, 1108, 1112, 1118, 1134, 1193, 1205, 1207, 1216, 1219, 1267, 1334, 1400, 1427, 1596, 1681]


### 기존 방법에 명시적 피드백(사용자가 평가한 영화 점수)을 추가해 실험
- 위 테스트 까지는 봣는지 안봤는지 0,1로만 구분했지만,
- rating 점수를 추가 기입

In [64]:
adj_matrix = np.ndarray(shape,dtype=int)
for user_id, movie_id, rating, time in raw_data:
    adj_matrix[user_id][movie_id] = rating
adj_matrix

array([[5, 3, 4, ..., 0, 0, 0],
       [4, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [5, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 5, 0, ..., 0, 0, 0]])

In [65]:
my_id, my_vector = 0, adj_matrix[0]
best_match, best_match_id, best_match_vector = 999,-1,[]

for user_id, user_vector in enumerate(adj_matrix):
    if my_id != user_id:
        euclidean_dist = np.sqrt(np.sum(np.square(my_vector - user_vector)))
        if euclidean_dist < best_match:
            best_match = euclidean_dist
            best_match_id = user_id
            best_match_vector = user_vector
            
print('Best Match : {}, Best Match ID : {}'.format(best_match,best_match_id))

Best Match : 55.06359959174482, Best Match ID : 737


In [66]:
my_id, my_vector = 0, adj_matrix[0]
best_match, best_match_id, best_match_vector = -1,-1,[]

for user_id, user_vector in enumerate(adj_matrix):
    if my_id != user_id:
        cos_similarity = compute_cos_similarity(my_vector, user_vector)
        if cos_similarity > best_match:
            best_match = cos_similarity
            best_match_id = user_id
            best_match_vector = user_vector
            
print('Best Match : {}, Best Match ID : {}'.format(best_match,best_match_id))

Best Match : 0.569065731527988, Best Match ID : 915


### 협업 필터링(Collaborative Filtering)
- 사용자가 항목의 유사성을 동시에 고려해 추천
- 기존에 내 관심사가 아닌 항목이라도 추천 가능
- 자동으로 임베딩 학습 가능
- 협업 필터링은 다음과 같은 장단점을 갖고 있다.
    - 장점
        - 자동으로 임베딜을 학습하기 때문에 도메인 지식이 필요 없다.
        - 기존의 관심사가 아니더라도 추천 가능
    - 단점
        - 학습 과정에 나오지 않은 항목은 임베딩을 만들 수 없음
        - 추가 특성을 사용하기 어려움

In [67]:
from surprise import KNNBasic, SVD, SVDpp, NMF
from surprise.model_selection import cross_validate

In [70]:
data = Dataset.load_builtin('ml-100k',prompt=False)

### KNN

In [72]:
model = KNNBasic()
cross_validate(model, data, measures=['rmse','mae'],cv=5,n_jobs=4,verbose=True)

Evaluating RMSE, MAE of algorithm KNNBasic on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9801  0.9761  0.9770  0.9886  0.9750  0.9794  0.0049  
MAE (testset)     0.7739  0.7722  0.7681  0.7825  0.7693  0.7732  0.0051  
Fit time          0.23    0.29    0.36    0.34    0.35    0.31    0.05    
Test time         3.73    4.37    4.57    3.81    3.00    3.90    0.55    


{'test_rmse': array([0.98009805, 0.97612454, 0.97698906, 0.98860583, 0.97503996]),
 'test_mae': array([0.77393691, 0.77224802, 0.76811837, 0.78254347, 0.76932767]),
 'fit_time': (0.2255690097808838,
  0.29337000846862793,
  0.3621032238006592,
  0.3433573246002197,
  0.34873127937316895),
 'test_time': (3.7257888317108154,
  4.3696489334106445,
  4.574054002761841,
  3.8096859455108643,
  2.9971249103546143)}

### SVD

In [73]:
model = SVD()
cross_validate(model, data, measures=['rmse','mae'],cv=5,n_jobs=4,verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9312  0.9389  0.9403  0.9316  0.9365  0.9357  0.0037  
MAE (testset)     0.7352  0.7429  0.7385  0.7368  0.7361  0.7379  0.0027  
Fit time          8.09    9.47    9.21    7.99    6.38    8.23    1.10    
Test time         0.31    0.28    0.21    0.14    0.14    0.22    0.07    


{'test_rmse': array([0.93119069, 0.93893163, 0.94026663, 0.93160746, 0.93652876]),
 'test_mae': array([0.73523136, 0.74287612, 0.73845163, 0.73678182, 0.73605637]),
 'fit_time': (8.094111680984497,
  9.467317819595337,
  9.214981079101562,
  7.9942238330841064,
  6.3757219314575195),
 'test_time': (0.3113830089569092,
  0.28177714347839355,
  0.20604801177978516,
  0.14030003547668457,
  0.13979005813598633)}

### NMF

In [74]:
model = NMF()
cross_validate(model, data, measures=['rmse','mae'],cv=5,n_jobs=4,verbose=True)

Evaluating RMSE, MAE of algorithm NMF on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9696  0.9539  0.9657  0.9569  0.9675  0.9627  0.0062  
MAE (testset)     0.7622  0.7496  0.7594  0.7522  0.7606  0.7568  0.0050  
Fit time          6.31    7.91    7.90    6.73    5.21    6.81    1.02    
Test time         0.31    0.19    0.17    0.11    0.12    0.18    0.07    


{'test_rmse': array([0.96963412, 0.95389285, 0.96567372, 0.95686821, 0.96749769]),
 'test_mae': array([0.76216308, 0.74961203, 0.75941243, 0.752216  , 0.76061043]),
 'fit_time': (6.3059937953948975,
  7.91035795211792,
  7.896173000335693,
  6.731925964355469,
  5.208373069763184),
 'test_time': (0.31101226806640625,
  0.18948101997375488,
  0.16993403434753418,
  0.11088895797729492,
  0.12450528144836426)}

### SVD++ (need long time)

In [75]:
model = SVDpp()
cross_validate(model, data, measures=['rmse','mae'],cv=5,n_jobs=4,verbose=True)

Evaluating RMSE, MAE of algorithm SVDpp on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9176  0.9236  0.9195  0.9160  0.9105  0.9174  0.0043  
MAE (testset)     0.7186  0.7234  0.7213  0.7197  0.7129  0.7192  0.0035  
Fit time          331.22  332.18  331.31  333.60  158.24  297.31  69.54   
Test time         5.16    5.26    5.00    2.82    2.42    4.13    1.24    


{'test_rmse': array([0.91761513, 0.92356056, 0.9194866 , 0.91601133, 0.91052808]),
 'test_mae': array([0.71864062, 0.72339963, 0.72126621, 0.7197061 , 0.71294894]),
 'fit_time': (331.2192270755768,
  332.17549896240234,
  331.3080530166626,
  333.6004853248596,
  158.2362880706787),
 'test_time': (5.15802001953125,
  5.25727391242981,
  4.997334003448486,
  2.8187379837036133,
  2.415692090988159)}

### 하이브리드(Hybrid)
- 컨텐츠 기반 필터링과 협업 필터링을 조합한 방식
- 많은 하이브리드 방식이 존재
- 실습에서는 협업 필터링으로 임베딩을 학습하고 컨텐츠 기반 필터링으로 유사도 기반 추천을 수행하는 추천 엔진 개발

In [55]:
data = Dataset.load_builtin('ml-100k',prompt=False)
raw_data = np.array(data.raw_ratings, dtype=int)
raw_data[:,0]-=1
raw_data[:,1]-=1

In [56]:
n_users = np.max(raw_data[:,0])
n_movies = np.max(raw_data[:,1])
shape = (n_users+1,n_movies+1)
shape

(943, 1682)

In [57]:
adj_matrix = np.ndarray(shape, dtype=int)
for user_id, movie_id, rating, time in raw_data:
    adj_matrix[user_id][movie_id] = rating

In [58]:
adj_matrix

array([[5, 3, 4, ..., 0, 0, 0],
       [4, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [5, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 5, 0, ..., 0, 0, 0]])

- 행렬 분해

In [59]:
U, S, V = randomized_svd(adj_matrix, n_components=2)
S = np.diag(S)

In [60]:
print(U.shape)
print(S.shape)
print(V.shape)

(943, 2)
(2, 2)
(2, 1682)


In [61]:
np.matmul(np.matmul(U,S),V)

array([[ 3.91732663e+00,  1.47276644e+00,  7.98261988e-01, ...,
         6.24907189e-04,  1.41100852e-02,  1.36545878e-02],
       [ 1.85777226e+00,  3.96191175e-01,  5.05705740e-01, ...,
         5.38862978e-03,  1.77237914e-03,  5.26968095e-04],
       [ 8.94989517e-01,  1.71578497e-01,  2.51738682e-01, ...,
         2.92094923e-03,  5.39937171e-04, -1.25733753e-04],
       ...,
       [ 9.92051955e-01,  2.10814957e-01,  2.70363365e-01, ...,
         2.89019297e-03,  9.34221962e-04,  2.66612193e-04],
       [ 1.30425401e+00,  5.27669941e-01,  2.50080165e-01, ...,
        -4.20677765e-04,  5.30525683e-03,  5.28069948e-03],
       [ 2.82999397e+00,  9.70812247e-01,  6.15871694e-01, ...,
         2.02091492e-03,  8.67740813e-03,  8.03107892e-03]])

- 사용자 기반 추천
- 나와 비슷한 취향을 가진 다른 사용자의 행동을 추천
- 사용자 특징 벡터의 유사도 사용

In [62]:
my_id, my_vector = 0, U[0]
best_match, best_match_id, best_match_vector = -1,-1,[]

for user_id, user_vector in enumerate(U):
    if my_id != user_id:
        cos_similarity = compute_cos_similarity(my_vector, user_vector)
        if cos_similarity > best_match:
            best_match = cos_similarity
            best_match_id = user_id
            best_match_vector = user_vector
            
print('Best Match : {}, Best Match ID : {}'.format(best_match,best_match_id))

Best Match : 0.9999942295956324, Best Match ID : 235


In [63]:
recommed_list = []
for i, log in enumerate(zip(adj_matrix[my_id], adj_matrix[best_match_id])):
    log1, log2 = log
    if log1<1. and log2>0.:
        recommed_list.append(i)
print(recommed_list)

[272, 273, 274, 281, 285, 288, 293, 297, 303, 306, 312, 317, 327, 332, 369, 410, 418, 419, 422, 426, 428, 431, 434, 442, 461, 475, 477, 482, 495, 503, 504, 505, 506, 509, 519, 520, 522, 525, 531, 545, 548, 590, 594, 595, 613, 631, 654, 658, 660, 672, 684, 685, 691, 695, 698, 704, 716, 728, 734, 749, 755, 863, 865, 933, 1012, 1038, 1101, 1327, 1400]


- 항목 기반 추천
- 내가 본 항목과 비슷한 항목을 추천
- 항목 특징 벡터의 유사도 사용

In [64]:
my_id, my_vector = 0, V.T[0]
best_match, best_match_id, best_match_vector = -1,-1,[]

for user_id, user_vector in enumerate(V.T):
    if my_id != user_id:
        cos_similarity = compute_cos_similarity(my_vector, user_vector)
        if cos_similarity > best_match:
            best_match = cos_similarity
            best_match_id = user_id
            best_match_vector = user_vector
            
print('Best Match : {}, Best Match ID : {}'.format(best_match,best_match_id))

Best Match : 0.9999999951364144, Best Match ID : 1287


In [65]:
recommed_list = []
for i, user_vector in enumerate(adj_matrix):
    if adj_matrix[i][my_id] > 0.9:
        recommed_list.append(i)
print(recommed_list)

[0, 1, 4, 5, 9, 12, 14, 15, 16, 17, 19, 20, 22, 24, 25, 37, 40, 41, 42, 43, 44, 48, 53, 55, 56, 57, 58, 61, 62, 63, 64, 65, 66, 69, 71, 72, 74, 76, 78, 80, 81, 82, 83, 88, 91, 92, 93, 94, 95, 96, 98, 100, 101, 105, 107, 108, 116, 119, 120, 123, 124, 127, 129, 130, 133, 136, 137, 140, 143, 144, 147, 149, 150, 156, 157, 159, 161, 167, 173, 176, 177, 180, 181, 183, 188, 192, 193, 197, 198, 199, 200, 201, 202, 203, 208, 209, 212, 215, 221, 222, 229, 230, 231, 233, 234, 241, 242, 243, 245, 246, 247, 248, 249, 250, 251, 252, 253, 255, 261, 262, 264, 267, 270, 273, 274, 275, 276, 278, 279, 285, 286, 288, 289, 290, 291, 292, 293, 294, 295, 296, 297, 298, 300, 302, 304, 306, 307, 310, 311, 312, 313, 319, 321, 323, 324, 325, 326, 329, 330, 331, 335, 337, 338, 339, 342, 343, 344, 346, 347, 349, 356, 358, 359, 362, 364, 370, 373, 377, 378, 379, 380, 386, 387, 388, 389, 392, 393, 394, 395, 397, 398, 400, 401, 402, 405, 406, 410, 411, 415, 416, 418, 421, 423, 424, 428, 431, 433, 434, 437, 440, 444, 

- 비음수 행렬 분해를 사용한 하이브리드 추천

In [67]:
adj_matrix

array([[5, 3, 4, ..., 0, 0, 0],
       [4, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [5, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 5, 0, ..., 0, 0, 0]])

In [68]:
A, B, iter = non_negative_factorization(adj_matrix,n_components=2)

/Users/psh/opt/anaconda3/lib/python3.7/site-packages/sklearn/decomposition/_nmf.py:1035: FutureWarning: The default value of init will change from random to None in 0.23 to make it consistent with decomposition.NMF.
  "with decomposition.NMF.", FutureWarning)


In [69]:
np.matmul(A,B)

array([[3.71104586e+00, 1.48477312e+00, 7.39553911e-01, ...,
        3.64524413e-03, 1.45528188e-02, 1.44126789e-02],
       [2.11740181e+00, 2.36761022e-01, 5.51611687e-01, ...,
        4.76686816e-03, 2.46791493e-05, 0.00000000e+00],
       [9.85430147e-01, 1.10187612e-01, 2.56717825e-01, ...,
        2.21848096e-03, 1.14855752e-05, 0.00000000e+00],
       ...,
       [1.04463709e+00, 1.16807941e-01, 2.72142030e-01, ...,
        2.35177247e-03, 1.21756554e-05, 0.00000000e+00],
       [1.45790214e+00, 5.42099472e-01, 2.99288129e-01, ...,
        1.61341874e-03, 5.15835211e-03, 5.10702876e-03],
       [2.44663874e+00, 9.41418379e-01, 4.95535829e-01, ...,
        2.56820530e-03, 9.08625531e-03, 8.99725661e-03]])

- 사용자 기반 추천

In [70]:
my_id, my_vector = 0, U[0]
best_match, best_match_id, best_match_vector = -1,-1,[]

for user_id, user_vector in enumerate(U):
    if my_id != user_id:
        cos_similarity = compute_cos_similarity(my_vector, user_vector)
        if cos_similarity > best_match:
            best_match = cos_similarity
            best_match_id = user_id
            best_match_vector = user_vector
            
print('Best Match : {}, Best Match ID : {}'.format(best_match,best_match_id))

Best Match : 0.9999942295956324, Best Match ID : 235


In [71]:
recommed_list = []
for i, log in enumerate(zip(adj_matrix[my_id], adj_matrix[best_match_id])):
    log1, log2 = log
    if log1<1. and log2>0.:
        recommed_list.append(i)
print(recommed_list)

[272, 273, 274, 281, 285, 288, 293, 297, 303, 306, 312, 317, 327, 332, 369, 410, 418, 419, 422, 426, 428, 431, 434, 442, 461, 475, 477, 482, 495, 503, 504, 505, 506, 509, 519, 520, 522, 525, 531, 545, 548, 590, 594, 595, 613, 631, 654, 658, 660, 672, 684, 685, 691, 695, 698, 704, 716, 728, 734, 749, 755, 863, 865, 933, 1012, 1038, 1101, 1327, 1400]


- 항목 기반 추천

In [83]:
my_id, my_vector = 0, V.T[0]
best_match, best_match_id, best_match_vector = -1,-1,[]

for user_id, user_vector in enumerate(V.T):
    if my_id != user_id:
        cos_similarity = compute_cos_similarity(my_vector, user_vector)
        if cos_similarity > best_match:
            best_match = cos_similarity
            best_match_id = user_id
            best_match_vector = user_vector
            
print('Best Match : {}, Best Match ID : {}'.format(best_match,best_match_id))

Best Match : 0.9999999951364144, Best Match ID : 1287


In [88]:
recommed_list = []
for i, user_vector in enumerate(adj_matrix):
    if adj_matrix[i][my_id] > 0.9:
        recommed_list.append(i)
print(recommed_list)

[0, 1, 4, 5, 9, 12, 14, 15, 16, 17, 19, 20, 22, 24, 25, 37, 40, 41, 42, 43, 44, 48, 53, 55, 56, 57, 58, 61, 62, 63, 64, 65, 66, 69, 71, 72, 74, 76, 78, 80, 81, 82, 83, 88, 91, 92, 93, 94, 95, 96, 98, 100, 101, 105, 107, 108, 116, 119, 120, 123, 124, 127, 129, 130, 133, 136, 137, 140, 143, 144, 147, 149, 150, 156, 157, 159, 161, 167, 173, 176, 177, 180, 181, 183, 188, 192, 193, 197, 198, 199, 200, 201, 202, 203, 208, 209, 212, 215, 221, 222, 229, 230, 231, 233, 234, 241, 242, 243, 245, 246, 247, 248, 249, 250, 251, 252, 253, 255, 261, 262, 264, 267, 270, 273, 274, 275, 276, 278, 279, 285, 286, 288, 289, 290, 291, 292, 293, 294, 295, 296, 297, 298, 300, 302, 304, 306, 307, 310, 311, 312, 313, 319, 321, 323, 324, 325, 326, 329, 330, 331, 335, 337, 338, 339, 342, 343, 344, 346, 347, 349, 356, 358, 359, 362, 364, 370, 373, 377, 378, 379, 380, 386, 387, 388, 389, 392, 393, 394, 395, 397, 398, 400, 401, 402, 405, 406, 410, 411, 415, 416, 418, 421, 423, 424, 428, 431, 433, 434, 437, 440, 444, 